# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 12 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [4]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (12 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [5]:
import re

def clean_split_sentence(sentence: str, bos: bool, eos: bool):
    if bos:
        sentence = '<s> ' + sentence
    if eos:
        sentence += ' </s>'
    
    sentence = re.sub(r'  ', ' ', sentence.strip(' ').lower())
    return sentence.split(' ')

def count_ngram_in_sentence(sentence: List[str], ngram: List[str]):
    ngram_len = len(ngram)
    return len([i for i in range(len(sentence) - ngram_len + 1) if sentence[i : i + ngram_len] == ngram])

def count_ngrams_for_sentence(sentence: List[str], order: int):
    res = {}
    
    for legth in range(1, order + 1):
        for i in range(len(sentence) - legth + 1):
            ngram = sentence[i : i + legth]
            count = count_ngram_in_sentence(sentence, ngram)
            res[tuple(ngram)] = count

    return res

def add_ngrams(current: Dict[Tuple[str], int], new: Dict[Tuple[str], int]):
    for ngram in new:
        if ngram in current:
            current[ngram] += new[ngram]
        else:
            current[ngram] = new[ngram]

    return current

# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = {}

    for sentence in train_text:
        sentence_ngrams = count_ngrams_for_sentence(clean_split_sentence(sentence, bos, eos), order)
        ngrams = add_ngrams(ngrams, sentence_ngrams)
        
    return ngrams

In [6]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [7]:
{1, 2, 3}.update({5, 6})

In [8]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    
    other_ngrams_count = sum([value for key, value in counts.items() if len(key) == len(ngram) and key[: -1] == ngram[: -1]])

    if V is None:
        V = len([key for key in counts if len(key) == 1])
    
    ngram_count = counts[ngram] if ngram in counts else 0

    # print("({0} + {1}) / ({2} + {1} * {3})".format(ngram_count, k, other_ngrams_count, V) )

    prob = (ngram_count + k) / (other_ngrams_count + k * V)
    return prob

In [9]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [10]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def fit(self, train_text: List[str]) -> None:
        self.ngrams_count = count_ngrams(train_text, self.order, self.bos, self.eos)
        self.vocab = [key[0] for key in self.ngrams_count if len(key) == 1]
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        return np.log(calculate_ngram_prob(ngram, self.ngrams_count, V=len(self.vocab), k=self.k))
           
    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = 0

        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        ngrams = [tuple(words[max(i - self.order, 0) : i]) for i in range(1, len(words) + 1)]
        # print(ngrams)
        logprob = np.mean([self.predict_ngram_log_proba(ngram) for ngram in ngrams])
        # print(logprob)

        return logprob
        
    def ppl(self, text: List[str]) -> float:
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        # 
        
        # print(text)

        log_prob_list = [self.predict_log_proba(sentence.split()) for sentence in text]
        # print(log_prob_list)
        mean_log_prob = np.mean(log_prob_list)
        perplexity = np.exp(-mean_log_prob)
        
        # print(perplexity)
        
        return perplexity

In [11]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (4 балла)

In [14]:
def predict_next_word(lm: NgramLM, prefix: str, topk=4):
    # функция, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix 
    vocab = lm.vocab
    
    probability = []
    for posible_next in vocab:
        posible_text = prefix + " " + posible_next
        proba = lm.ppl(posible_text.split())
        probability.append([posible_next, proba])

    probability = np.array(probability)
    top_prob_args = np.array(np.argpartition(probability, kth=-topk, axis=0))[-topk:, 1]
    top_prob = probability[top_prob_args]
    
    return top_prob
    

Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).

Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 

In [15]:
#Your code here  
test_text = ["Языковые модели",
"Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.",
"В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении \"по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас\", биграмами будут \"по-моему мы\", \"мы сэкономим\", \"сэкономим уйму\" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.",
"Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. ",
"Практическая работа разделена на 2 части: ",
"1. Построение нграмой языковой модели - основная часть, 12 баллов",
"1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла",
"Построение нграмной языковой модели. (12 баллов)",
"Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: ",
"В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:",
"Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке",
"Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. ",
"Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа]. Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.",
"Формула сглаживания Лапласа выглядит следующим образом:",
"Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм",
"Основной метрикой язковых моделей является перплексия. ",
"Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку."
]

def clear_text(text: List[str]):
    res = []
    for sentence in text:
        clean_sentence = re.sub(r'[^\w\s]', '', sentence.lower())
        clean_sentence = re.sub(r'  ', ' ', clean_sentence).strip()
        res.append(clean_sentence)
    return res

clean_text = clear_text(test_text)

lm = NgramLM(order=2)
lm.fit(clean_text)

display(predict_next_word(lm, "по-моему"))
display(predict_next_word(lm, "сэкономим"))
display(predict_next_word(lm, "нграм"))
display(predict_next_word(lm, "вероятности"))

display(predict_next_word(lm, "нграм имеет"))

array([['нулю', '94.00117664258929'],
       ['языковые', '94.15121953901945'],
       ['выборке', '94.22579489882729'],
       ['в', '94.88404281461804']], dtype='<U32')

array([['нулю', '94.22796257875743'],
       ['языковые', '94.37836746660042'],
       ['выборке', '94.45312274588792'],
       ['в', '95.11295874147878']], dtype='<U32')

array([['текста', '90.2446692434014'],
       ['вероятность', '90.45826957968208'],
       ['нграм', '90.45826957968208'],
       ['модели', '90.73920377342874']], dtype='<U32')

array([['нулю', '94.15267041885308'],
       ['языковые', '94.30295512677755'],
       ['выборке', '94.37765067340938'],
       ['в', '95.03695943190489']], dtype='<U32')

array([['текста', '97.23727235686678'],
       ['вероятность', '97.39064599072992'],
       ['нграм', '97.39064599072992'],
       ['модели', '97.59218437355459']], dtype='<U32')